In [1]:
import pandas as pd
import numpy as np

In [2]:
df_main = pd.read_csv('clean_data.csv')
df_imdb = pd.read_csv('clean_imdb.csv')

In [3]:
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, RidgeClassifier, RidgeClassifierCV
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, ComplementNB

In [4]:
np.random.seed(42)
pdf_main = df_main.reindex(np.random.permutation(df_main.index))
pdf_imdb = df_imdb.reindex(np.random.permutation(df_imdb.index))

In [6]:
X_train, y_train = pdf_imdb['clean_text'], pdf_imdb['class']

In [7]:
X_test, y_test = pdf_main['clean_text'], pdf_main['class']

In [9]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Flatten, Conv1D, MaxPooling1D, Dropout
from keras.optimizers import Adam

In [12]:
import os
embeddings_index = {}
f = open(os.path.join('../', 'glove.6B.200d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [14]:
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(X_train)

seq_train = tokenizer.texts_to_sequences(X_train)
seq_test = tokenizer.texts_to_sequences(X_test)

In [34]:
EMBEDDING_DIM = 200
MAX_SEQUENCE_LENGTH = 150
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [35]:
embedding_layer = Embedding(len(tokenizer.word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

W0726 07:33:16.840545 139907794396992 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [37]:
pad_train = pad_sequences(seq_train, maxlen=MAX_SEQUENCE_LENGTH)
pad_test = pad_sequences(seq_test, maxlen=MAX_SEQUENCE_LENGTH)

In [43]:
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(128, dropout=.2, recurrent_dropout=.2, return_sequences=True))
model.add(LSTM(128, dropout=.2, recurrent_dropout=.2, return_sequences=True))
model.add(LSTM(128, dropout=.2, recurrent_dropout=.2))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

W0726 07:34:26.095335 139907794396992 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 07:34:26.134574 139907794396992 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0726 07:34:26.155071 139907794396992 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0726 07:34:26.155964 139907794396992 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please us

In [45]:
model.fit(pad_train, y_train, epochs=2, batch_size=64, validation_data=(pad_test, y_test))

Train on 50000 samples, validate on 2000 samples
Epoch 1/2
50000/50000 [==============================] - 528s 11ms/step - loss: 0.4631 - acc: 0.7830 - val_loss: 0.5161 - val_acc: 0.7805
Epoch 2/2
50000/50000 [==============================] - 511s 10ms/step - loss: 0.3542 - acc: 0.8478 - val_loss: 0.3880 - val_acc: 0.8215


In [47]:
model.fit(pad_train, y_train, epochs=5, batch_size=64, validation_data=(pad_test, y_test))

Train on 50000 samples, validate on 2000 samples
Epoch 1/5
50000/50000 [==============================] - 497s 10ms/step - loss: 0.3152 - acc: 0.8670 - val_loss: 0.3540 - val_acc: 0.8520
Epoch 2/5
18496/50000 [==========>...................] - ETA: 5:03 - loss: 0.2936 - acc: 0.8764

KeyboardInterrupt: 

In [48]:
model.save("bigger_rnn.hdf5")